In [25]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/side_concordance_results.xlsx')

In [ ]:
def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    
    if 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'
    
preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)

NameError: name 'preop_imaging_df' is not defined

In [27]:
def check_concordance(row):
    imaging = row['Sestamibi']
    surgical = row['Surgical_Findings']
    
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'
    imaging_glands = [g.strip() for g in imaging.split(',')]
    surgical_glands = [g.strip() for g in surgical.split(',')]
    
    if any(sg in imaging_glands for sg in surgical_glands):
        return 'Yes (Exact)'
    
    imaging_sides = []
    for ig in imaging_glands:
        if 'Left' in ig:
            imaging_sides.append('Left')
        if 'Right' in ig:
            imaging_sides.append('Right')
    
    surgical_sides = []
    for sg in surgical_glands:
        if 'Left' in sg:
            surgical_sides.append('Left')
        if 'Right' in sg:
            surgical_sides.append('Right')
    
    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'
    
    return 'Incorrect Localization'


In [30]:
# Define modality
modality = 'Sestamibi'

# Apply extract_side function if not already done to get imaging sides
preop_imaging_df['Sestamibi Imaging Side'] = preop_imaging_df[modality].apply(extract_side)

# Filter to scans that localized anywhere (Left, Right)
localized_sestamibi = preop_imaging_df[preop_imaging_df['Sestamibi Imaging Side'].isin(['Left', 'Right'])]

# Count total localized scans
total_localized_sestamibi = localized_sestamibi.shape[0]
total_localized_sestamibi



113

In [31]:
# Compare sides to see if there is at least a side match
# First extract surgical sides if not already done
preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)

# Filter to only rows in localized_4dct for side match calculation
localized_sestamibi = localized_sestamibi.copy()
localized_sestamibi['Side Match'] = localized_sestamibi['Sestamibi Imaging Side'] == localized_sestamibi['Surgical Side']

# Count how many matched on side (including exact matches)
side_match_count_sestamibi = localized_sestamibi['Side Match'].sum()

# Calculate percentage of scans that correlated with surgery (side match or exact match both count as a win here)
side_concordance_percentage_sestamibi = (side_match_count_sestamibi / total_localized_sestamibi) * 100 if total_localized_sestamibi > 0 else None

# Create a results dataframe for clean display
side_concordance_sestamibi_df = pd.DataFrame({
    'Metric': [
        'Total Sestamibi scans with localization',
        'Scans matching surgical side or exact gland'
    ],
    'Value': [
        total_localized_sestamibi,
        f"{side_match_count_sestamibi} ({side_concordance_percentage_sestamibi:.2f}%)"
    ]
})

# Display the dataframe
display(side_concordance_sestamibi_df)

,Metric,Value
0,Total Sestamibi scans with localization,113
1,Scans matching surgical side or exact gland,108 (95.58%)
